Global Runoff Data Center (GRDC) https://portal.grdc.bafg.de/

Download process:
1. Pick sub-region(s) or station(s) of interest.
2. Submit data request form. Select to include watershed boundaries.
3. Download zip file from request email response.

Download components:
- stationbasins.geojson: metadata and geoemetry of the watershed upstream of the gauge.
- subregions.geojson: global geodataframe of subregions
- stationID_Q_.txt: time series of Q (m3/s) for each station

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import codebase

In [2]:
datadir = '/global/scratch/users/ann_scheliga/aux_dam_datasets/GRDC_CRB/'

In [3]:
subregions_meta = gpd.read_file(datadir + 'subregions.geojson')
stations_meta = gpd.read_file(datadir + 'stationbasins.geojson')

In [4]:
subregions_meta.head()

,GmlID,FID,OBJECTID,WMOBB,SUBREGNUM,SUBREGNAME,SUBREG_DES,SUM_SUB_AR,REGNUM,REGNAME,RIVERBASIN,OCEANNUM,OCEAN,geometry
0,hydroreg_portal.0,0,1,101.0,1011.0,MEDJERDA (also MAJARDAH),Medjerda river basin,23171.5,1,Africa,MEDJERDA (also MAJARDAH),2,Atlantic Ocean,"MULTIPOLYGON (((10.17917 37.12083, 10.13333 37..."
1,hydroreg_portal.1,1,2,102.0,1021.0,MELRHIR,Chott Melrhir interior basin,146405,1,Africa,MELRHIR,9,---,"MULTIPOLYGON (((1.97917 33.99583, 1.95933 34.0..."
2,hydroreg_portal.2,2,3,102.0,1022.0,RHARSA,Chott el Rharsa interior basin,114355.5,1,Africa,RHARSA,9,---,"MULTIPOLYGON (((7.19167 35.37917, 7.12696 35.3..."
3,hydroreg_portal.3,3,4,103.0,1031.0,DJERID,Chott el Djerid interior basin,59021,1,Africa,DJERID,9,---,"MULTIPOLYGON (((10.11667 32.43750, 10.14498 32..."
4,hydroreg_portal.4,4,5,104.0,1040.0,"Mediterranean Sea (ex 161, 627, 628, 691, 693)",basins draining into Mediterranean Sea between...,102164.9,1,Africa,---,2,Atlantic Ocean,"MULTIPOLYGON (((-1.53253 35.23700, -1.49238 35..."


In [5]:
test_meta = pd.read_table(datadir+'4152050_Q_Day.Cmd.txt', nrows=36, encoding='unicode_escape')
test_meta

,# Title: GRDC STATION DATA FILE
0,# --------------
1,# Format: DOS-ASCII
2,# Field delimiter: ;
3,# missing values are indicated by -999.000
4,#
5,# file generation date: 2024-12-04
6,#
7,# GRDC-No.: 4152050
8,# River: COLORADO RIVER
9,# Station: BELOW YUMA MAIN CANAL...


In [6]:
test_df = pd.read_table(datadir+'4152050_Q_Day.Cmd.txt', header = 36, encoding='unicode_escape',delimiter=';')
test_df.tail()

,YYYY-MM-DD,hh:mm,Value
22033,2024-01-27,--:--,45.024
22034,2024-01-28,--:--,36.246
22035,2024-01-29,--:--,33.697
22036,2024-01-30,--:--,32.564
22037,2024-01-31,--:--,33.414


In [7]:
test_df = pd.read_table(datadir+'4152050_Q_Day.Cmd.txt', header = 36, encoding='unicode_escape',delimiter=';')
test_df['YYYY-MM-DD'] = pd.to_datetime(test_df['YYYY-MM-DD'])
test_df.rename(columns = {'YYYY-MM-DD':'Date',' Value':'Q m3s'}, inplace =True)
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df.drop(columns='hh:mm',inplace=True)
test_df.set_index('Date',inplace=True)
test_df[
    (test_df.index >= pd.to_datetime('2018-01-01')) &
    (test_df.index < pd.to_datetime('2024-01-01'))]

,Q m3s
Date,
2018-01-01,25.542
2018-01-02,28.600
2018-01-03,27.920
2018-01-04,38.794
2018-01-05,39.927
...,...
2023-12-27,22.512
2023-12-28,19.624
2023-12-29,19.057


In [8]:
test_df[
    (test_df.index >= pd.to_datetime('2018-01-01')) &
    (test_df.index < pd.to_datetime('2024-01-01'))]

,Q m3s
Date,
2018-01-01,25.542
2018-01-02,28.600
2018-01-03,27.920
2018-01-04,38.794
2018-01-05,39.927
...,...
2023-12-27,22.512
2023-12-28,19.624
2023-12-29,19.057


In [10]:
codebase.load_data.load_GRDC_timeseries('4152050_Q_Day.Cmd.txt')

,Q m3s
Date,
2000-01-01,24.777
2000-01-02,30.016
2000-01-03,37.661
2000-01-04,24.749
2000-01-05,23.107
...,...
2024-01-27,45.024
2024-01-28,36.246
2024-01-29,33.697
